In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("final_df.csv")
df.drop('Unnamed: 0',axis = 1,inplace =  True)

In [3]:
df.sample(5)

,hotel,arrival_date_month,meal,market_segment,distribution_channel,reserved_room_type,deposit_type,customer_type,is_canceled,lead_time,...,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,adr,required_car_parking_spaces,total_of_special_requests,year,month,day
96310,0.417738,0.391911,0.375296,0.368061,0.410861,0.392770,0.284846,0.409328,0,31,...,0,0,1,9.0,135.9,0,2,2016,9,10
107665,0.417738,0.323999,0.375296,0.611240,0.410861,0.392770,0.284846,0.254333,0,116,...,0,0,0,171.0,90.0,0,0,2017,3,26
35430,0.279828,0.397434,0.375296,0.611240,0.410861,0.318521,0.284846,0.254333,0,202,...,0,0,0,251.0,62.0,0,1,2017,5,5
49081,0.417738,0.408589,0.375296,0.368061,0.410861,0.392770,0.284846,0.409328,0,113,...,0,0,0,9.0,96.3,0,1,2016,4,17
43199,0.417738,0.391911,0.375296,0.611240,0.410861,0.392770,0.284846,0.254333,1,83,...,0,0,0,1.0,60.0,0,0,2015,9,17


In [4]:
x=df.drop('is_canceled',axis=1)
y=df['is_canceled']

In [5]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [6]:
# select a suitable alpha (equivalent of penalty).
# The bigger the alpha the less features that will be selected.
model = Lasso(alpha=0.005, random_state=0)
feature_sel_model = SelectFromModel(model) # remember to set the seed, the random state in this function
feature_sel_model.fit(x,y)

SelectFromModel(estimator=Lasso(alpha=0.005, random_state=0))

In [15]:
feature_sel_model.get_support()

array([False, False, False, False, False, False,  True, False,  True,
        True, False,  True,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [21]:
selected_features = x.columns[(feature_sel_model.get_support())]
selected_features 

Index(['deposit_type', 'lead_time', 'arrival_date_week_number',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults',
       'previous_cancellations', 'previous_bookings_not_canceled',
       'booking_changes', 'agent', 'adr', 'required_car_parking_spaces',
       'total_of_special_requests', 'year', 'month', 'day'],
      dtype='object')

In [22]:
print(f'total features {x.shape[1]}')
print(f'selected features {len(selected_features)}')

total features 27
selected features 16


In [25]:
x = x[selected_features]
x.head()

,deposit_type,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,adr,required_car_parking_spaces,total_of_special_requests,year,month,day
0,0.284846,342,27,0,0,2,0,0,3,86.545532,0.0,0,0,2015,7,1
1,0.284846,737,27,0,0,2,0,0,4,86.545532,0.0,0,0,2015,7,1
2,0.284846,7,27,0,1,1,0,0,0,86.545532,75.0,0,0,2015,7,2
3,0.284846,13,27,0,1,1,0,0,0,304.000000,75.0,0,0,2015,7,2
4,0.284846,14,27,0,2,2,0,0,0,240.000000,98.0,0,1,2015,7,3


### splitting dataset & model Building

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
x_train, x_test, y_train , y_test = train_test_split(x,y,test_size = 0.2 ,random_state = 0 )

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
lr = LogisticRegression()

In [32]:
lr.fit(x_train,y_train)

C:\Users\hp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [33]:
y_pred = lr.predict(x_test)

In [35]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)
score

0.7152362503158427

In [36]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[12710,  2184],
       [ 4578,  4274]], dtype=int64)

### Cross validate your model

In [45]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(lr,x,y,cv=10)

C:\Users\hp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\hp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [46]:
score

array([0.61012381, 0.69333783, 0.69679104, 0.59412112, 0.76139139,
       0.64238187, 0.64288722, 0.7140571 , 0.73955526, 0.58962264])

In [47]:
score.mean()

0.6684269281386135

In [40]:
#fit naive bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [41]:
### classifier models
models = []
models.append(('LogisticRegression', LogisticRegression()))
models.append(('Naive Bayes',GaussianNB()))
models.append(('RandomForest', RandomForestClassifier()))
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier(n_neighbors = 5)))

In [48]:
models

[('LogisticRegression', LogisticRegression()),
 ('Naive Bayes', GaussianNB()),
 ('RandomForest', RandomForestClassifier()),
 ('Decision Tree', DecisionTreeClassifier()),
 ('KNN', KNeighborsClassifier())]

In [42]:


for name, model in models:
    print(name)
    model.fit(x_train, y_train)
    
    # Make predictions.
    predictions = model.predict(x_test)

    # Compute the error.
    from sklearn.metrics import confusion_matrix
    print(confusion_matrix(predictions, y_test))

    from sklearn.metrics import accuracy_score
    print(accuracy_score(predictions,y_test))
    print('\n')

LogisticRegression


C:\Users\hp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[[12710  4578]
 [ 2184  4274]]
0.7152362503158427


Naive Bayes
[[6956 1304]
 [7938 7548]]
0.6107976080181925


RandomForest
[[14783  1027]
 [  111  7825]]
0.9520761391392234


Decision Tree
[[14223   639]
 [  671  8213]]
0.9448328139476122


KNN
[[13318  2748]
 [ 1576  6104]]
0.8179061736713552




NameError: name 'RandomForest' is not defined